# IMPORTING MODULES

In [ ]:
#importing libraries / modules

%matplotlib inline
import matplotlib.pyplot as plt
import csv
import pandas
import sklearn
import pickle
import PIL
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.tree import DecisionTreeClassifier 
sklearn.model_selection.learning_curve

import warnings
warnings.filterwarnings('ignore')

# PREPARING DATA

In [ ]:
#reading files using pandas
data1 = pd.read_csv('spam.csv', encoding='latin-1')
data2 = pd.read_csv('spamraw.csv', encoding='latin-1')

In [ ]:
#dropinG null values in data1
data1 = data1.drop(["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], axis=1)
data1 = data1.rename(columns={"v1" : "label", "v2":"text"})
data2 = data2.rename(columns={"type" : "label", "text":"text"})

In [ ]:
#concatenating data1 and data2 as one data
pieces = [data1, data2]
spam_df = pd.concat(pieces, ignore_index=True)

spam_df.head(10)

In [ ]:
#checking the shape of data; rows and columns
spam_df.shape

In [ ]:
#checking the number of data for each type of text
spam_df['label'].value_counts()

In [ ]:
#Replacing spam and ham index with 1 and 0 respectivelyS
spam_df = spam_df.replace(['ham','spam'],[0, 1])

In [ ]:
spam_df[5560:5570]

# PLOTTING GRAPHICAL REPRESENTATIONS

In [ ]:
spam_df['length'] = spam_df['text'].map(lambda x: len(x))
spam_df.hist(column='length',bins=50, figsize=(8,5))

In [ ]:
spam_df.hist(column='length', by='label', bins=100, figsize=(15,5))

In [ ]:
#Numbers of all words in the text
text = " ".join(review for review in spam_df.text)
print ("There are {} words in the combination of all text.".format(len(text)))

DISPLAY TEXT IN WORDCLOUD TO SHOW MOST OCCURING WORDS IN TEXT

In [ ]:
# Create stopword list:
stopwords = set(STOPWORDS)
stopwords.update()

In [ ]:
# Generate a word cloud image
wordcloud = WordCloud(stopwords=stopwords, background_color="white").generate(text)

In [ ]:
# Display the generated image:
plt.figure( figsize=(8,8), facecolor='yellow')
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.tight_layout(pad=2)
plt.show()

In [ ]:
#representing the percentage distribution of using a pie chart

spam_df['label'].value_counts().plot(kind='pie',figsize = (8,5), shadow=True, autopct='%1.2f%%')


# PROCESSING TEXT

In [ ]:
import string
def text_process(text):
    
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = [word for word in text.split() if word.lower() not in stopwords.words('english')]
    
    return " ".join(text)

In [ ]:
spam_df['text'] = spam_df['text'].apply(text_process)

In [ ]:
#displaying the last 10 rows to confirm
spam_df.tail(10)

In [ ]:
label = pd.DataFrame(spam_df['label'])
text = pd.DataFrame(spam_df['text'])

In [ ]:
#  how many times a unique word appears in the dataset
from collections import Counter

total_counts = Counter()
for i in range(len(text)):
    for word in text.values[i][0].split(" "):
        total_counts[word] += 1

print("Total words in data set: ", len(total_counts))

In [ ]:
## Sorting in decreasing order (Word with highest frequency appears first)
vocab = sorted(total_counts, key=total_counts.get, reverse=True)
print(vocab[:100])

In [ ]:
# Mapping from words to index
vocab_size = len(vocab)
word2idx = {}

#print vocab_size
for i, word in enumerate(vocab):
    word2idx[word] = i

In [ ]:
# Text to Vector
def text_to_vector(text):
    word_vector = np.zeros(vocab_size)
    for word in text.split(" "):
        if word2idx.get(word) is None:
            continue
        else:
            word_vector[word2idx.get(word)] += 1
    return np.array(word_vector)

In [ ]:
# Convert all titles to vectors
word_vectors = np.zeros((len(text), len(vocab)), dtype=np.int_)
for ii, (_, text_) in enumerate(text.iterrows()):
    word_vectors[ii] = text_to_vector(text_[0])

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(spam_df['text'])
vectors.shape

In [ ]:
#features = word_vectors
features = vectors

#spliting the data set to train and test data
from sklearn.model_selection import train_test_split
trainingSet, testSet = train_test_split(spam_df, test_size=0.2)
print ("Training Set: ", len(trainingSet))
print ("Test Set: ", len(testSet))



In [ ]:
X_trainingSet, X_testSet, y_trainingSet, y_testSet = sklearn.model_selection.train_test_split(features, spam_df['label'], test_size=0.15, random_state=111)

print (X_trainingSet.shape)
print (X_testSet.shape)
print (y_trainingSet.shape)
print (y_testSet.shape)

# TRAINING MODELS

In [ ]:
import xgboost as xgb

# Set our parameters for xgboost
params = {}
params['objective'] = 'binary:logistic'
params['eval_metric'] = 'error'
params['eta'] = 0.02
params['max_depth'] = 10

In [ ]:
#training trainingSet

d_train = xgb.DMatrix(X_trainingSet, label=y_trainingSet)

watchlist = [(d_train, 'train')]

bst = xgb.train(params, d_train, 400, watchlist, early_stopping_rounds=50, verbose_eval=10)

In [ ]:
#training testSet

d_train = xgb.DMatrix(X_testSet, label=y_testSet)

watchlist = [(d_train, 'train')]

bst = xgb.train(params, d_train, 400, watchlist, early_stopping_rounds=50, verbose_eval=10)

PREDICTION ACCURACY OF THE MODEL

In [ ]:
from sklearn.metrics import accuracy_score

# Predict values for test set
d_testSet = xgb.DMatrix(X_testSet)
p_testSet = bst.predict(d_testSet)

# Apply function round() to each element in np array
# so predictions are all either 0 or 1.
npround = np.vectorize(round)
p_test_ints = npround(p_testSet)

# Error rate for test set
accuracy = accuracy_score(y_testSet, p_test_ints)
print("Test Accuracy: ", accuracy)

# CLASSIFICATION METHODS

In [ ]:
#finding accuracy using different classification methods

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:
#Classification method with respective parameters;loss funx, optimizers, activation funx

svc = SVC(kernel='sigmoid', gamma=1.0)
knc = KNeighborsClassifier(n_neighbors=49)
mnb = MultinomialNB(alpha=0.2)
dtc = DecisionTreeClassifier(min_samples_split=7, random_state=111)
lrc = LogisticRegression(solver='liblinear', penalty='l1')
rfc = RandomForestClassifier(n_estimators=31, random_state=111)

In [ ]:
clfs = {'SVC' : svc,'KN' : knc, 'NB': mnb, 'DT': dtc, 'LR': lrc, 'RF': rfc}

In [ ]:
def train(clf, features, targets):    
    clf.fit(features, targets)

def predict(clf, features):
    return (clf.predict(features))

In [ ]:
pred_scores_word_vectors = []
for k,v in clfs.items():
    train(v, X_trainingSet, y_trainingSet)
    pred = predict(v, X_testSet)
    pred_scores_word_vectors.append((k, [accuracy_score(y_testSet , pred)]))

In [ ]:
def calc(gr_truth, predicted):
    return normalized_mutual_info_score(gr_truth, predicted) 

In [ ]:
#Prediction accuracy of the various classifiers
predictions_word_vectors = pd.DataFrame.from_items(pred_scores_word_vectors,orient='index', columns=['Score_'])
predictions_word_vectors

In [ ]:
# Plotting a graphical display of the accuracyn scores
predictions_word_vectors.plot(kind='bar', ylim=(0.85,1.0), figsize=(9,6), align='center', colormap="Accent")
plt.xticks(np.arange(6), predictions_word_vectors.index)
plt.ylabel('Accuracy Score')
plt.title('Distribution by Classifier')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

plt.savefig('accuracy.png')

In [ ]:
predictions_word_vectors.plot(kind='bar', ylim=(0.85,1.0), figsize=(9,6), align='center', colormap="Accent")
plt.xticks(np.arange(6), predictions_word_vectors.index)
plt.ylabel('Accuracy Score')
plt.title('Distribution by Classifier - Word Vectors')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

plt.savefig('accuracy1.png')

In [ ]:
# tokenizing sample message
from nltk.tokenize import word_tokenize
text1 = "WINNER!! You just won a free ticket to Bahamas. Send your Details"
tokens =word_tokenize(text1)
print(tokens)

In [ ]:
def find(p):
    if p == 1:
        print ("WARNING! This is a SPAM")
    else:
        print ("Geniune Message")

In [ ]:
text = ["You just need a free ticket to Bahamas!"]
integers = vectorizer.transform(text)

In [ ]:
p = mnb.predict(integers)[0]
find(p)

In [ ]:
text = ['You are awarded a SiPix Digital Camera! call 08061451071 from landline.']
integers = vectorizer.transform(text)

In [ ]:
post = mnb.predict(integers)[0]
find(post)

In [ ]:
from sklearn.externals import joblib
joblib.dump(mnb, 'best.pkl')
print ("Best Model (Naive Bayes) Saved")

In [ ]:
best = joblib.load('best.pkl')

In [ ]:
best